In [1]:
#%% md

# ライブラリ

#%%

import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym
from collections import namedtuple
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F

#%% md

# 動画に保存する関数

#%%

from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def save_as_gif(frames):
    plt.figure(figsize=(frames[0].shape[1]/72.0, frames[0].shape[0]/72.0), dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')

    def animate(i):
        patch.set_data(frames[i])

    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=50)

    anim.save('DQN-CartPole.mp4')
    display(display_animation(anim, default_mode='loop'))

#%%

Transition = namedtuple('Transition', 'state action next_state reward')

# 学習に使う変数を整理
ENV = 'CartPole-v0'
# 報酬割引率
GAMMA = 0.99
# 1試行（1エピソード）の最大ステップ数
MAX_STEP = 200
# 最大試行回数（エピソード数）
NUM_EPISODES = 1000
# バッチサイズ
BATCH_SIZE = 32
# キャパ
CAPACITY = 10000

#%%

# ミニバッチ学習のための経験データを保存するクラス
class ReplayMemory:

    def __init__(self, CAPACITY):
        # メモリ容量
        self.capacity = CAPACITY
        # 経験を保存する
        self.memory = []
        # 保存場所を示す変数
        self.index = 0

    # 経験をメモリに保存する
    def push(self, state, action, next_state, reward):

        # メモリの長さが指定したキャパシティ以下の場合にNoneを追加しておく
        if len(self.memory) < self.capacity:
            self.memory.append(None)

        # 経験をメモリに保存
        self.memory[self.index] = Transition(state, action, next_state, reward)

        # インデックスを1ずらす
        self.index = (self.index + 1) % self.capacity

    # 指定したバッチサイズ分，ランダムに経験を取り出す
    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    # memoryの長さを返す
    def __len__(self):
        return len(self.memory)

#%%

# エージェントが行う行動を与えられた状態によって判断する部分（深層強化学習（DQN）を行う部分）
class Brain:

    def __init__(self, num_states, num_actions):
        self.num_actions = num_actions
        self.memory = ReplayMemory(CAPACITY)

        # ニューラルネットワーク
        self.model = nn.Sequential()
        self.model.add_module('fc1', nn.Linear(num_states, 32))
        self.model.add_module('relu1', nn.ReLU())
        self.model.add_module('fc2', nn.Linear(32, 32))
        self.model.add_module('relu2', nn.ReLU())
        self.model.add_module('fc3', nn.Linear(32, num_actions))

        print(self.model)

        # 最適化手法
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.0001)

    # 結合パラメータを学習する部分
    def replay(self):

        # 最初にメモリサイズを確認する
        # 指定したバッチサイズより小さい場合は何もしない
        if len(self.memory) < BATCH_SIZE:
            return

        # ミニバッチ用のデータを取得（ランダム）
        transitions = self.memory.sample(BATCH_SIZE)

        # transitions は (state, action, next_state, reward) * BATCH_SIZE
        # (state * BATCH_SIZE, action * BATCH_SIZE, next_state * BATCH_SIZE, reward * BATCH_SIZE)
        batch = Transition(*zip(*transitions))

        state_batch = torch.cat(batch.state)
        action_batch = torch.cat(batch.action)
        reward_batch = torch.cat(batch.reward)
        non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])

        # 教師信号となるQ(s_t, a_t)を求める
        # モデルを推論モードに
        self.model.eval()

        # self.model(state_batch)は，2つのQ値を出力する
        # [torch.FloatTensor of size BATCH_SIZE * 2]になってるので
        # 実行したアクション（a_t）に対応するQ値をaction_batchで行った行動a_tのindexを使って取得する
        state_action_values = self.model(state_batch).gather(1, action_batch)

        # CartPole がdoneになっていない，かつ，next_stateがあるかをチェックするためのマスクを作成
        non_final_mask = torch.ByteTensor(tuple(map(lambda s: s is not None, batch.next_state)))

        # maxQ(s_t+1, a)を求める
        next_state_values = torch.zeros(BATCH_SIZE)

        # 次の状態があるindexの最大Q値を求める
        next_state_values[non_final_mask] = self.model(non_final_next_states).max(1)[0].detach()

        # Q学習の行動価値関数更新式からQ(S_t, a_t)を求める
        expected_state_action_values = reward_batch + GAMMA * next_state_values

        # モデルを訓練モードに切り替え
        self.model.train()

        # 二乗誤差の代わりにHuber関数を使う
        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

        # 勾配をリセット
        self.optimizer.zero_grad()
        # 誤差逆伝搬
        loss.backward()
        # ニューラルネットワークの重み更新
        self.optimizer.step()

    # 現在の状態に応じて行動を決定する
    def decide_action(self, state, episode):
        # ε-greedy法で徐々に最適行動を採用するようにする
        epsilon = 0.5 * (1 / (episode + 1))

        if epsilon <= np.random.uniform(0, 1):
            # 推論モードに
            self.model.eval()
            # ネットワークの出力の最大値のindexを取得
            # view関数で行列サイズを（1 * 1）に調整
            with torch.no_grad():
                action = self.model(state).max(1)[1].view(1, 1)

        else:
            # 右，左ランダムに行動する
            # actionは[torch.LongTensor of size 1 * 1]
            action = torch.LongTensor([[random.randrange(self.num_actions)]])

        return action

#%%

# エージェントクラス
class Agent:
    def __init__(self, num_states, num_actions):
        # Brainクラスをインスタンス化
        self.brain = Brain(num_states, num_actions)

    # Q関数の更新
    def update_q_function(self):
        self.brain.replay()

    # アクションを決定する
    def get_action(self, state, episode):
        action = self.brain.decide_action(state, episode)
        return action

    # 状態を保存
    def memorize(self, state, action, next_state, reward):
        self.brain.memory.push(state, action, next_state, reward)

#%%

# CartPoleを実行する環境クラス
class Environment:

    def __init__(self):
        self.env = gym.make(ENV)
        num_states = self.env.observation_space.shape[0]
        num_actions = self.env.action_space.n
        self.agent = Agent(num_states, num_actions)

    def run(self):

        episode_10_list = np.zeros(10) # 10試行分の立ち続けた平均ステップ数の出力に使う
        complete_episodes = 0  # 195step以上連続で立ち続けた試行数
        is_episode_final = False  # 最終試行フラグ
        frames = []  # 動画用に画像を格納する変数

        # 全エピソードループ
        for episode in range(NUM_EPISODES):
            # エピソード毎に環境を初期化
            observation = self.env.reset()

            state = observation

            # numpyからpytorchのテンソルに変換
            state = torch.from_numpy(state).type(torch.FloatTensor)

            # size を 1*4 に変換
            state = torch.unsqueeze(state, 0)

            for step in range(MAX_STEP):

                # 最終試行はframesに画像を追加しておく
                if is_episode_final:
                    frames.append(self.env.render(mode='rgb_array'))

                # 最初の行動を決める
                action = self.agent.get_action(state, episode)

                # 最初の行動から次の状態を求める
                observation_next, _, done, _ = self.env.step(action.item())

                # 報酬を与える
                if done:
                    # 次の状態はないのでNoneを代入
                    state_next = None

                    # 直前10エピソードで立てた平均ステップ数を格納
                    episode_10_list = np.hstack((episode_10_list[1:], step + 1))

                    if step < 195:
                        # こけたら報酬-1
                        reward = torch.FloatTensor([-1.0])
                        # 連続成功回数を0にリセット
                        complete_episodes = 0
                    else:
                        # 立ったまま終了した場合は報酬1
                        reward = torch.FloatTensor([1.0])
                        # 連続成功回数をインクリメント
                        complete_episodes += 1
                else:
                    # 途中の報酬は0
                    reward = torch.FloatTensor([0.0])
                    state_next = observation_next
                    state_next = torch.from_numpy(state_next).type(torch.FloatTensor)
                    state_next = torch.unsqueeze(state_next, 0)

                # メモリに経験を追加
                self.agent.memorize(state, action, state_next, reward)

                # Q関数をニューラルネットで更新
                self.agent.update_q_function()

                # 状態を次の状態に更新
                state = state_next

                # エピソード終了時
                if done:
                    print('{0}エピソード: {1}ステップで終了'.format(episode, step + 1))
                    break

            # 最終エピソードの場合は動画を保存
            if is_episode_final:
                save_as_gif(frames)
                break

            # 10回連続で成功したら、次のエピソードで終わりにする
            if complete_episodes >= 10:
                print('10回連続成功')
                is_episode_final = True


#%%

cartpole_env = Environment()
cartpole_env.run()

#%%




Sequential(
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)
0エピソード: 11ステップで終了
1エピソード: 10ステップで終了
2エピソード: 9ステップで終了
3エピソード: 10ステップで終了
4エピソード: 10ステップで終了
5エピソード: 10ステップで終了
6エピソード: 11ステップで終了
7エピソード: 10ステップで終了
8エピソード: 10ステップで終了
9エピソード: 10ステップで終了
10エピソード: 10ステップで終了
11エピソード: 28ステップで終了
12エピソード: 29ステップで終了
13エピソード: 20ステップで終了
14エピソード: 34ステップで終了
15エピソード: 34ステップで終了
16エピソード: 42ステップで終了
17エピソード: 23ステップで終了
18エピソード: 33ステップで終了
19エピソード: 17ステップで終了
20エピソード: 20ステップで終了
21エピソード: 35ステップで終了
22エピソード: 45ステップで終了
23エピソード: 28ステップで終了
24エピソード: 77ステップで終了
25エピソード: 127ステップで終了
26エピソード: 19ステップで終了
27エピソード: 27ステップで終了
28エピソード: 9ステップで終了
29エピソード: 11ステップで終了
30エピソード: 16ステップで終了
31エピソード: 10ステップで終了
32エピソード: 10ステップで終了
33エピソード: 19ステップで終了
34エピソード: 40ステップで終了
35エピソード: 12ステップで終了
36エピソード: 21ステップで終了
37エピソード: 13ステップで終了
38エピソード: 10ステップで終了
39エピソード: 37ステップで終了
40エピソード: 10ステップで終了
41エピ